In [1]:
!python -V

Python 3.9.16


In [2]:
!which python

/home/lucapug/mlops-zoomcamp/cohorts/2023/02-experiment-tracking/.venv/bin/python


In [3]:
import pandas as pd

In [4]:
import pickle

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [7]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/home/lucapug/mlops-zoomcamp/cohorts/2023/02-experiment-tracking/notebooks/mlruns/1', creation_time=1684854067199, experiment_id='1', last_update_time=1684854067199, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [8]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')

In [10]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
#untracked lr without regularization model (lr)

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715204122913

In [15]:
with open('../models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [14]:
with mlflow.start_run() as lr_lasso:

    mlflow.set_tag("developer", "luca")
    mlflow.set_tag("model","Lasso")

    mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "../data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    
    lr_lasso = Lasso(alpha)
    lr_lasso.fit(X_train, y_train)

    y_pred = lr_lasso.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    # here below one way to save models (the other is log_model)
    # (but I am not sure is the right model cause lin_reg.bin was for lr simple (no Lasso))
    #mlflow.log_artifact(local_path="../models/lin_reg.bin", artifact_path="models_pickle")
    
    #my version...
    with open('../models/lin_reg_lasso.bin', 'wb') as f_out:
        pickle.dump((dv, lr_lasso), f_out)
        
    mlflow.log_artifact(local_path="../models/lin_reg_lasso.bin", artifact_path="models_pickle")

In [16]:
import xgboost as xgb

In [19]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [17]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [21]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [22]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[18:23:54] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:11.42610                          
[1]	validation-rmse:8.14446                           
[2]	validation-rmse:7.19372                           
[3]	validation-rmse:6.91127                           
[4]	validation-rmse:6.81747                           
[5]	validation-rmse:6.77965                           
[6]	validation-rmse:6.76385                           
[7]	validation-rmse:6.75499                           
[8]	validation-rmse:6.74886                           
[9]	validation-rmse:6.73879                           
[10]	validation-rmse:6.73511                          
[11]	validation-rmse:6.73114                          
[12]	validation-rmse:6.72959                          
[13]	validation-rmse:6.72761                          
[14]	validation-rmse:6.72824                          
[15]	validation-rmse:6.72277                          
[

In [23]:
#the below params are copied from mlflow dashboard lowest rmse = 6.286

best_params = {
    'learning_rate': 0.08345717715305563,
    'max_depth': 37,
    'min_child_weight': 1.1860754367078694,
    'objective': 'reg:linear',
    'reg_alpha': 0.06625811875705244,
    'reg_lambda': 0.35910116532678693,
    'seed': 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

2023/05/24 22:05:09 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '9ab0244de06f456d8a9c935ea3c5079b', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[22:05:09] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.69879
[1]	validation-rmse:18.34015
[2]	validation-rmse:17.10797
[3]	validation-rmse:15.99181
[4]	validation-rmse:14.98089
[5]	validation-rmse:14.06909
[6]	validation-rmse:13.24690
[7]	validation-rmse:12.50651
[8]	validation-rmse:11.84330
[9]	validation-rmse:11.24802
[10]	validation-rmse:10.71493
[11]	validation-rmse:10.23790
[12]	validation-rmse:9.81374
[13]	validation-rmse:9.43650
[14]	validation-rmse:9.10197
[15]	validation-rmse:8.80426
[16]	validation-rmse:8.54056
[17]	validation-rmse:8.30835
[18]	validation-rmse:8.10192
[19]	validation-rmse:7.91862
[20]	validation-rmse:7.75638
[21]	validation-rmse:7.61290
[22]	validation-rmse:7.48749
[23]	validation-rmse:7.37482
[24]	validation-rmse:7.27726
[25]	validation-rmse:7.19039
[26]	validation-rmse:7.11324
[27]	validation-rmse:7.04481
[28]	validation-rmse:6.98440
[29]	validation-rmse:6.93173
[30]	vali

2023/05/24 22:07:11 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2023/05/24 22:07:16 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/lucapug/mlops-zoomcamp/cohorts/2023/02-experiment-tracking/.venv/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [18]:
mlflow.xgboost.autolog(disable=True)

#model management: save, versioning,..

with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.08345717715305563,
        'max_depth': 37,
        'min_child_weight': 1.1860754367078694,
        'objective': 'reg:linear',
        'reg_alpha': 0.06625811875705244,
        'reg_lambda': 0.35910116532678693,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("../models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("../models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[17:29:08] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.69879
[1]	validation-rmse:18.34015
[2]	validation-rmse:17.10797
[3]	validation-rmse:15.99181
[4]	validation-rmse:14.98089
[5]	validation-rmse:14.06909
[6]	validation-rmse:13.24690
[7]	validation-rmse:12.50651
[8]	validation-rmse:11.84330
[9]	validation-rmse:11.24802
[10]	validation-rmse:10.71493
[11]	validation-rmse:10.23790
[12]	validation-rmse:9.81374
[13]	validation-rmse:9.43650
[14]	validation-rmse:9.10197
[15]	validation-rmse:8.80426
[16]	validation-rmse:8.54056
[17]	validation-rmse:8.30835
[18]	validation-rmse:8.10192
[19]	validation-rmse:7.91862
[20]	validation-rmse:7.75638
[21]	validation-rmse:7.61290
[22]	validation-rmse:7.48749
[23]	validation-rmse:7.37482
[24]	validation-rmse:7.27726
[25]	validation-rmse:7.19039
[26]	validation-rmse:7.11324
[27]	validation-rmse:7.04481
[28]	validation-rmse:6.98440
[29]	validation-rmse:6.93173
[30]	vali

/home/lucapug/mlops-zoomcamp/cohorts/2023/02-experiment-tracking/.venv/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [19]:
logged_model = 'runs:/1148a33d50974481b3845d573316df07/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

loaded_model

[17:58:36] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 1148a33d50974481b3845d573316df07

In [20]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

[17:59:05] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [21]:
xgboost_model

In [22]:
y_pred = xgboost_model.predict(valid)

In [23]:
y_pred[:10]

array([14.763997 ,  7.1031647, 15.94838  , 24.256823 ,  9.324065 ,
       17.188349 , 11.341891 ,  8.262217 ,  9.009202 , 20.598415 ],
      dtype=float32)

In [24]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("../models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/home/lucapug/mlops-zoomcamp/cohorts/2023/02-experiment-tracking/.venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
